Data sources: [Japan](https://www.ipss.go.jp/p-toukei/JMD/00/index-en.html), [Tokyo](https://www.ipss.go.jp/p-toukei/JMD/13/index-en.html), [Kyoto](https://www.ipss.go.jp/p-toukei/JMD/26/index-en.html) → "Life Tables"<br>
wiki > [List of Japanese prefectures by life expectancy](https://en.wikipedia.org/wiki/List_of_Japanese_prefectures_by_life_expectancy) <i>[(alternative)](https://en.wikipedia.org/wiki/User:Lady3mlnm/List_of_Japanese_prefectures_by_life_expectancy_(alternative))</i> / [Продолжительность жизни в префектурах Японии](https://ru.wikipedia.org/wiki/Продолжительность_жизни_в_префектурах_Японии)<br />
[Demographics of Japan](https://en.wikipedia.org/wiki/Demographics_of_Japan#Life_expectancy) [Demographics of Tokyo](https://en.wikipedia.org/wiki/Demographics_of_Tokyo#Life_expectancy)<br>

[selection of colors](http://mal-bioit.ru/survey-web-colors)<br />
see also: [Nippon.com](https://www.nippon.com/en/search.html?s=life%20expectancy)

In [2]:
import pandas as pd
import math
import re
from collections import namedtuple

In [3]:
YEARS = (2010, 2020, 2023)  # the tuple has to contains 3 years
assert len(YEARS) == 3

AGES = [0, 1] + list(range(5, 111, 5))
WRITE_TABLE = True
NMB_REGION = 0       # number for Japan
# NMB_REGION = 13      # number for Tokyo

In [4]:
# This block of code extracts name of region from CSV-file with data.
# This name is used only to name correctly file with resuls of the script work

pattern_region_general = re.compile(r'(?<=^[\d.]{3}).*(?=(.ken|.to|.fu), Life tables)')  # what is in the most cases, like '47.Okinawa.ken'
pattern_region_1       = re.compile(r'(?<=^[\d.]{3}).*(?=, Life tables)')  # case of '01.Hokkaido' in the file Jp_01.csv
pattern_region_0       = re.compile(r'(?<=^).*(?=, Life tables)')  # case of 'Japan' in the file Jp_01.csv

def extract_name_of_region_from_file(region_nmb):
    with open(f"data-Life_tables/JpLT_{region_nmb:02}-total.csv", 'r') as fh:
        title_line = fh.readline()
        if region_nmb >= 2:
            fragment_name_found = re.search(pattern_region_general, title_line)
        elif region_nmb == 1:
            fragment_name_found = re.search(pattern_region_1, title_line)
        elif region_nmb == 0:
            fragment_name_found = re.search(pattern_region_0, title_line)
        else:
            raise ValueError(f"Not expected region number: {region_nmb}")
        name = fragment_name_found.group() if fragment_name_found else 'region name is not found'
    return name


name_region = extract_name_of_region_from_file(NMB_REGION)
print(f"Region #{NMB_REGION} - {name_region}")

Region #0 - Japan


<br>

In [6]:
def read_single_csv(file_name, year, ages):
    df_t = pd.read_csv(f"data-Life_tables/{file_name}", sep='\t', usecols=['Year', 'Age', 'ex'], skiprows=2)
    df_t = df_t.loc[df_t['Year'] == year] \
               .drop(columns='Year') \
               .set_index('Age')
    df_t = df_t.rename(index={'110+': '110'}).rename(index=lambda st: int(st))
    df_t = df_t.loc[df_t.index.isin(ages)]
    return df_t

# load data for region
df_total_0   = read_single_csv(f'JpLT_{NMB_REGION:02}-total.csv', year=YEARS[0], ages=AGES)
df_males_0   = read_single_csv(f'JpLT_{NMB_REGION:02}-males.csv', year=YEARS[0], ages=AGES)
df_females_0 = read_single_csv(f'JpLT_{NMB_REGION:02}-females.csv', year=YEARS[0], ages=AGES)

df_total_1   = read_single_csv(f'JpLT_{NMB_REGION:02}-total.csv', year=YEARS[1], ages=AGES)
df_males_1   = read_single_csv(f'JpLT_{NMB_REGION:02}-males.csv', year=YEARS[1], ages=AGES)
df_females_1 = read_single_csv(f'JpLT_{NMB_REGION:02}-females.csv', year=YEARS[1], ages=AGES)

df_total_2   = read_single_csv(f'JpLT_{NMB_REGION:02}-total.csv', year=YEARS[2], ages=AGES)
df_males_2   = read_single_csv(f'JpLT_{NMB_REGION:02}-males.csv', year=YEARS[2], ages=AGES)
df_females_2 = read_single_csv(f'JpLT_{NMB_REGION:02}-females.csv', year=YEARS[2], ages=AGES)

df = pd.concat([df_males_0, df_total_0, df_females_0,
                df_males_1, df_total_1, df_females_1,
                df_males_2, df_total_2, df_females_2], axis='columns')
df.columns = ['y0_males', 'y0_total', 'y0_females',
              'y1_males', 'y1_total', 'y1_females',
              'y2_males', 'y2_total', 'y2_females']
df.index.name = ''

# calculate sex difference
df.insert(loc=3,  column='y0_fΔm', value= (df['y0_females'] - df['y0_males']).round(2))
df.insert(loc=7,  column='y1_fΔm', value= (df['y1_females'] - df['y1_males']).round(2))
df.insert(loc=11,  column='y2_fΔm', value= (df['y2_females'] - df['y2_males']).round(2))

df.loc[[0, 1, 5, 100, 105, 110]]

,y0_males,y0_total,y0_females,y0_fΔm,y1_males,y1_total,y1_females,y1_fΔm,y2_males,y2_total,y2_females,y2_fΔm
,,,,,,,,,,,,
0,79.51,82.95,86.24,6.73,81.58,84.71,87.73,6.15,81.10,84.14,87.16,6.06
1,78.71,82.14,85.43,6.72,80.73,83.86,86.89,6.16,80.25,83.29,86.31,6.06
5,74.79,78.22,81.49,6.70,76.78,79.91,82.92,6.14,76.30,79.35,82.37,6.07
100,2.09,2.38,2.43,0.34,2.19,2.52,2.58,0.39,2.08,2.36,2.40,0.32
105,1.61,1.73,1.74,0.13,1.65,1.79,1.80,0.15,1.58,1.69,1.69,0.11
110,1.34,1.36,1.36,0.02,1.34,1.39,1.39,0.05,1.30,1.33,1.33,0.03


<br>
<br>

In [8]:
dd_replacement = {
    'Japan'     : {'jp': ('日本', ''), 'en': ('Japan', ''), 'ru': ('Япония', '')},
    'Hokkaido'  : {'jp': ('北海道', ''), 'en': ('Hokkaido', 'Hokkaido'), 'ru': ('Хокка́йдо', 'Хоккайдо')},
    'Aomori'    : {'jp': ('青森県', ''), 'en': ('Aomori ', 'Aomori Prefecture'), 'ru': ('Ао́мори', 'Аомори (префектура)')},
    'Iwate'     : {'jp': ('岩手県', ''), 'en': ('Iwate', 'Iwate Prefecture'), 'ru': ('Ива́те', 'Ивате (префектура)')},
    'Miyagi'    : {'jp': ('宮城県', ''), 'en': ('Miyagi', 'Miyagi Prefecture'), 'ru': ('Мия́ги', 'Мияги (префектура)')},
    'Akita'     : {'jp': ('秋田県', ''), 'en': ('Akita', 'Akita Prefecture'), 'ru': ('А́кита', 'Акита (префектура)')},
    'Yamagata'  : {'jp': ('山形県', ''), 'en': ('Yamagata', 'Yamagata Prefecture'), 'ru': ('Ямага́та', 'Ямагата (префектура)')},
    'Fukushima' : {'jp': ('福島県', ''), 'en': ('Fukushima', 'Fukushima Prefecture'), 'ru': ('Фуку́сима', 'Фукусима (префектура)')},
    'Ibaraki'   : {'jp': ('茨城県', ''), 'en': ('Ibaraki', 'Ibaraki Prefecture'), 'ru': ('Ибара́ки', 'Ибараки')},
    'Tochigi'   : {'jp': ('栃木県', ''), 'en': ('Tochigi', 'Tochigi Prefecture'), 'ru': ('Тоти́ги', 'Тотиги (префектура)')},
    'Gumma'     : {'jp': ('群馬県', ''), 'en': ('Gunma', 'Gunma Prefecture'), 'ru': ('Гумма', 'Гумма (префектура)')},
    'Saitama'   : {'jp': ('埼玉県', ''), 'en': ('Saitama', 'Saitama Prefecture'), 'ru': ('Са́йтама', 'Сайтама (префектура)')},
    'Chiba'     : {'jp': ('千葉県', ''), 'en': ('Chiba', 'Chiba Prefecture'), 'ru': ('Тиба', 'Тиба (префектура)')},
    'Tokyo'     : {'jp': ('東京都', ''), 'en': ('Tokyo', 'Tokyo'), 'ru': ('То́кио', 'Токио')},
    'Kanagawa'  : {'jp': ('神奈川県', ''), 'en': ('Kanagawa', 'Kanagawa Prefecture'), 'ru': ('Канага́ва', 'Канагава')},
    'Niigata'   : {'jp': ('新潟県', ''), 'en': ('Niigata', 'Niigata Prefecture'), 'ru': ('Ниига́та', 'Ниигата (префектура)')},
    'Toyama'    : {'jp': ('富山県', ''), 'en': ('Toyama', 'Toyama Prefecture'), 'ru': ('Тоя́ма', 'Тояма (префектура)')},
    'Ishikawa'  : {'jp': ('石川県', ''), 'en': ('Ishikawa', 'Ishikawa Prefecture'), 'ru': ('Исика́ва', 'Исикава')},
    'Fukui'     : {'jp': ('福井県', ''), 'en': ('Fukui', 'Fukui Prefecture'), 'ru': ('Фукуи́', 'Фукуи (префектура)')},
    'Yamanashi' : {'jp': ('山梨県', ''), 'en': ('Yamanashi', 'Yamanashi Prefecture'), 'ru': ('Ямана́си', 'Яманаси (префектура)')},
    'Nagano'    : {'jp': ('長野県', ''), 'en': ('Nagano', 'Nagano Prefecture'), 'ru': ('Нага́но', 'Нагано (префектура)')},
    'Gifu'      : {'jp': ('岐阜県', ''), 'en': ('Gifu', 'Gifu Prefecture'), 'ru': ('Ги́фу', 'Гифу (префектура)')},
    'Shizuoka'  : {'jp': ('静岡県', ''), 'en': ('Shizuoka', 'Shizuoka Prefecture'), 'ru': ('Сидзуо́ка', 'Сидзуока (префектура)')},
    'Aichi'     : {'jp': ('愛知県', ''), 'en': ('Aichi', 'Aichi Prefecture'), 'ru': ('А́йти', 'Айти (префектура)')},
    'Mie'       : {'jp': ('三重県', ''), 'en': ('Mie', 'Mie Prefecture'), 'ru': ('Ми́э', 'Миэ')},
    'Shiga'     : {'jp': ('滋賀県', ''), 'en': ('Shiga', 'Shiga Prefecture'), 'ru': ('Си́га', 'Сига')},
    'Kyoto'     : {'jp': ('都府', ''), 'en': ('Kyoto', 'Kyoto Prefecture'), 'ru': ('Кио́то', 'Киото (префектура)')},
    'Osaka'     : {'jp': ('大阪府', ''), 'en': ('Osaka', 'Osaka Prefecture'), 'ru': ('О́сака', 'Осака (префектура)')},
    'Hyogo'     : {'jp': ('兵庫県', ''), 'en': ('Hyōgo', 'Hyōgo Prefecture'), 'ru': ('Хио́го', 'Хиого')},
    'Nara'      : {'jp': ('奈良県', ''), 'en': ('Nara', 'Nara Prefecture'), 'ru': ('На́ра', 'Нара (префектура)')},
    'Wakayama'  : {'jp': ('和歌山県', ''), 'en': ('Wakayama', 'Wakayama Prefecture'), 'ru': ('Вакая́ма', 'Вакаяма (префектура)')},
    'Tottori'   : {'jp': ('鳥取県', ''), 'en': ('Tottori', 'Tottori Prefecture'), 'ru': ('Тотто́ри', 'Тоттори (префектура)')},
    'Shimane'   : {'jp': ('島根県', ''), 'en': ('Shimane', 'Shimane Prefecture'), 'ru': ('Сима́не', 'Симане')},
    'Okayama'   : {'jp': ('岡山県', ''), 'en': ('Okayama', 'Okayama Prefecture'), 'ru': ('Окаяма', 'Окаяма (префектура)')},
    'Hiroshima' : {'jp': ('広島県', ''), 'en': ('Hiroshima', 'Hiroshima Prefecture'), 'ru': ('Хироси́ма', 'Хиросима (префектура)')},
    'Yamaguchi' : {'jp': ('山口県', ''), 'en': ('Yamaguchi', 'Yamaguchi Prefecture'), 'ru': ('Ямагу́ти', 'Ямагути (префектура)')},
    'Tokushima' : {'jp': ('徳島県', ''), 'en': ('Tokushima', 'Tokushima Prefecture'), 'ru': ('Токуси́ма', 'Токусима (префектура)')},
    'Kagawa'    : {'jp': ('香川県', ''), 'en': ('Kagawa', 'Kagawa Prefecture'), 'ru': ('Кагава', 'Кагава (префектура)')},
    'Ehime'     : {'jp': ('愛媛県', ''), 'en': ('Ehime', 'Ehime Prefecture'), 'ru': ('Эхиме́', 'Эхиме')},
    'Kochi'     : {'jp': ('高知県', ''), 'en': ('Kōchi', 'Kōchi Prefecture'), 'ru': ('Ко́ти', 'Коти (префектура)')},
    'Fukuoka'   : {'jp': ('福岡県', ''), 'en': ('Fukuoka', 'Fukuoka Prefecture'), 'ru': ('Фукуо́ка', 'Фукуока (префектура)')},
    'Saga'      : {'jp': ('佐賀県', ''), 'en': ('Saga', 'Saga Prefecture'), 'ru': ('Са́га', 'Сага (префектура)')},
    'Nagasaki'  : {'jp': ('長崎県', ''), 'en': ('Nagasaki', 'Nagasaki Prefecture'), 'ru': ('Нагаса́ки', 'Нагасаки (префектура)')},
    'Kumamoto'  : {'jp': ('熊本県', ''), 'en': ('Kumamoto', 'Kumamoto Prefecture'), 'ru': ('Кумамо́то', 'Кумамото (префектура)')},
    'Oita'      : {'jp': ('大分県', ''), 'en': ('Ōita', 'Ōita Prefecture'), 'ru': ('О́ита', 'Оита (префектура)')},
    'Miyazaki'  : {'jp': ('宮崎県', ''), 'en': ('Miyazaki', 'Miyazaki Prefecture'), 'ru': ('Миядза́ки', 'Миядзаки (префектура)')},
    'Kagoshima' : {'jp': ('鹿児島県', ''), 'en': ('Kagoshima', 'Kagoshima Prefecture'), 'ru': ('Каго́сима', 'Кагосима (префектура)')},
    'Okinawa'   : {'jp': ('沖縄県', ''), 'en': ('Okinawa', 'Okinawa Prefecture'), 'ru': ('Окина́ва', 'Окинава (префектура)')}
}

In [9]:
def create_table(df, file_header, lang='en'):

    def if_value(x, prec=2):
        # decoration of regular values
        return '—' if math.isnan(x) else \
               f"{x:0.{prec}f}"  if x>=0 else \
               f"−{-x:0.{prec}f}"

    with open('design/' + file_header, mode='r', encoding="utf-8") as fh:
        table_header = fh.read()

    st = ''
    for age in df.index:
        ser = df.loc[age]

        st += '\n' + '|-\n' + \
              f'|style="padding-right:2ex;"| {age} ' + \
              f'||style="background:#e0ffd8;"| {if_value(ser["y0_total"])} ' + \
              f'||style="background:#eaf3ff;"| {if_value(ser["y0_males"])} ' + \
              f'||style="background:#fee7f6;"| {if_value(ser["y0_females"])} ' + \
              f'|| {if_value(ser["y0_fΔm"])} ' + \
              f'||style="background:#e0ffd8;border-left-width:2px;"| {if_value(ser["y1_total"])} ' + \
              f'||style="background:#eaf3ff;"| {if_value(ser["y1_males"])} ' + \
              f'||style="background:#fee7f6;"| {if_value(ser["y1_females"])} ' + \
              f'|| {if_value(ser["y1_fΔm"])} ' + \
              f'||style="background:#e0ffd8;border-left-width:2px;"| {if_value(ser["y2_total"])} ' + \
              f'||style="background:#eaf3ff;"| {if_value(ser["y2_males"])} ' + \
              f'||style="background:#fee7f6;"| {if_value(ser["y2_females"])} ' + \
              f'|| {if_value(ser["y2_fΔm"])}'


    if lang == 'ru':
        st = re.sub('(?<=\\d)\\.(?=\\d)', ',', st)  # replace . to comma, if this . is between two digits

    st = table_header + st + '\n|}'
    return st
        

table_code = create_table(df, file_header='single_region_year_detailed_ru.txt', lang='ru')
with open(f"output/Table code for single region -{name_region} -ru.txt", 'w', encoding="utf-8") as fh:
    fh.write(table_code)

table_code = create_table(df, file_header='single_region_year_detailed_en.txt', lang='en')
with open(f"output/Table code for single region -{name_region} -en.txt", 'w', encoding="utf-8") as fh:
    fh.write(table_code)